# VinBigData Chest X-ray Abnormalities Detection

## Model Ensembling Method

**Author: Théo LANGÉ** - s394369 - theo.lange.369@cranfield.ac.uk


This notebook describes the Model Ensembling Method applied to merge all the different prediction obtained through Yolov8 models. In addition, the use of a 2-class filter is shown.

It requires as input all the prediction files made with the notebook `AI_VinBigData_Yolov8` and the binary classification prediction made with the notebook `AI_VinBigData_ResNet101`. One could also access the all the prediction files directly on kaggle: 
> https://www.kaggle.com/datasets/theolange/ai-vinbigdata

For this notebook to work outside of kaggle, you will need to update the last cell in the `Setup` section to precise the path of all the needed directory. 

This notebook will first merge all the different prediction of the Yolov8 models through an ensembling method called Weighted Boxes Fusion. Then, a 2 class filter will be applied on the Prediction resultig from the ensembling Method

## Table of contents
0. [Libraries and Setup](#0)
1. [Yolov8 Ensembling](#1)
2. [2-Class Filter](#2)

<a id="0"></a>
# 0. Libraries and Setup

In [1]:
pip install -q ensemble_boxes

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import shutil
from glob import glob

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn as sns

from ensemble_boxes import *

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Define the root data directory
DATA_DIR = "/kaggle/input/ai-vinbigdata"

# Define the paths to the training and testing dicom folders respectively
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")
LABELS_DIR = os.path.join(DATA_DIR, "labels")
PRED_DIR = os.path.join(DATA_DIR, "Prediction_Files")

# Define paths to the relevant csv files
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
TEST_CSV = os.path.join(DATA_DIR, "test_meta.csv")

# Working directory
WORKING_DIR = "/kaggle/working"

<a id="1"></a>
# 1. Yolov8 Ensembling

In this section, all the Yolov8 predictions will be merged using a ensembling method.

In [4]:
"""
This function convert the labels, boxes and scores arrays into a string of the right format for submission

Input: a list of labels, boxes and scores
Output: A Prediction String in the right format
"""

def format_prediction_string(labels, boxes, scores):
    
    pred_strings = []
    for j in zip(labels, scores, boxes):
        pred_strings.append("{0} {1:.4f} {2} {3} {4} {5}".format(
            int(j[0]), j[1], int(j[2][0]), int(j[2][1]), int(j[2][2]), int(j[2][3])))

    return " ".join(pred_strings)

In [5]:
"""
Transform a Prediction String into a list of labels, a list of bounding boxes and a list of confidence score

Input: a Prediction String, the height and width of the corresponding image 
Output: The list of labels, the list of bounding boxes and the list of confidence score
"""

def str_2_box(PredictionString, h, w):
    
    predString = PredictionString.split(' ')
    
    boxes = []
    label = []
    score = []
    
    # Loop over all the predicted boxes (6 values)
    for i in range (0, len(predString), 6):
        # Retrieve the label
        label.append(int(predString[i]))
        
        # Retrieve the confidence score
        score.append(float(predString[i+1]))

        # Retrieve the bounding boxe and its relative position
        boxe = []
        boxe.append(int(predString[i+2])/w)
        boxe.append(int(predString[i+3])/h)

        boxe.append(int(predString[i+4])/w)
        boxe.append(int(predString[i+5])/h)

        boxes.append(boxe)
    
    return label, boxes, score

In [6]:
"""
This function will merge all Yolov8 prediction using the WBF algorithm

Input: None
Output: Prediction DataFrame obtains through an ensembling method of all Yolov8 prediction
"""

def ensemble_prediction_yolov8():
    
    # Get the location of the Yolov8 prediction file
    prediction_files = glob(os.path.join(PRED_DIR, 'submission_Yolov8x*.csv'))
    
    # Merge all the Yolov8 prediction DataFrames
    pred_df = pd.read_csv(prediction_files[0]).rename(columns={'PredictionString': 'PredictionString_0'})
    for idx, file in enumerate(prediction_files[1:]):
        pred_df = pred_df.merge(pd.read_csv(file).rename(columns={'PredictionString': f'PredictionString_{idx+1}'}), on='image_id', how='left')
        
    # Get the Test DataFrame for the image shape
    test_meta = pd.read_csv(TEST_CSV)
    
    preds = []
    img_size = test_meta.values
    
    # Loop over all the images and get the bounding boxes from every Prediction String
    for idx, row in pred_df.iterrows():
        img_id = row.image_id
        
        # Get the image height and width
        index = np.where(img_size[:,0] == img_id)[0][0]
        h, w = img_size[index,1], img_size[index,2]

        boxes = []
        labels = []
        scores = []
        
        # Get the list of each predicted bounding boxes predicted by the 5 Yolov8 models
        for i in range (len(prediction_files)):
            label, boxe, score = str_2_box(row[f'PredictionString_{i}'], h, w)
            labels.append(label)
            boxes.append(boxe)
            scores.append(score)

        # Merge all the predictions using the WBF algorithm
        boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=0.4, skip_box_thr=0.0001)

        # Retrieve the absolute position of the bounding boxes given the image height and width
        for i in range (len(boxes)):
            if labels[i] != 14:
                boxes[i][0] *= w
                boxes[i][2] *= w

                boxes[i][1] *= h
                boxes[i][3] *= h
                
            else:
                boxes[i][0] = 0
                boxes[i][2] = 1

                boxes[i][1] = 0
                boxes[i][3] = 1


        # Add the merged prediction to a list
        pred = {
                    'image_id': img_id,
                    'PredictionString': format_prediction_string(labels, boxes, scores)
        }
        preds.append(pred)
        
    # Create a DataFrame for the merged prediction
    pred_df = pd.DataFrame(preds)
    
    return pred_df

In [7]:
# Ensemble all the Yolov8 models and get the submission file
ensemble_pred_df = ensemble_prediction_yolov8()
ensemble_pred_df.head()

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 0.8000 0 0 1 1 3 0.0686 778 1115 1974 1695
1,004f33259ee4aef671c2b95d54e4be68,0 0.4410 1083 568 1551 960
2,008bdde2af2462e86fd373a445d0f4cd,3 0.8587 1091 1412 1930 1780 0 0.8381 1422 820...
3,009bc039326338823ca3aa84381f17f1,3 0.8289 660 1052 1554 1348 0 0.6013 994 475 1...
4,00a2145de1886cb9eb88869c85d74080,3 0.8483 772 1288 1861 1639 0 0.7149 1121 708 ...


In [8]:
ensemble_pred_df.to_csv("ensemble_Yolov8.csv", index=None)

<a id="2"></a>
# 2. 2-Class Filter

Apply a 2-class filter to the ensembled prediction.

In [9]:
pred_2cls = pd.read_csv(os.path.join(PRED_DIR, "Resnet101_2cls_pred.csv")) 
pred_2cls.head()

,image_id,Finding,No Finding
0,002a34c58c5b758217ed1f584ccbcfe9,0.000005,0.999995
1,004f33259ee4aef671c2b95d54e4be68,0.000108,0.999892
2,008bdde2af2462e86fd373a445d0f4cd,0.983205,0.016795
3,009bc039326338823ca3aa84381f17f1,0.000928,0.999072
4,00a2145de1886cb9eb88869c85d74080,0.988707,0.011293


In [10]:
"""
This function will apply a 2-class filter to the ensembled prediction

Input: Prediction DataFrame after ensembling method, Prediction DataFrame for 2-class filter, three threshold for the filter
Output: Final Submission DataFrame after 2-class Filter 
"""

def apply_2cls_filter(ensemble_pred_df, pred_2cls, low_threshold = 0.0, intermediate_threshold = 0.995, high_threshold = 0.995):
    
    # Merge the two DataFrame
    merged_df = pd.merge(ensemble_pred_df, pred_2cls, on="image_id", how="left")
    
    # For each images
    for i in range(len(merged_df)):
        # Get the confidence score for the image being normal
        p0 = merged_df.loc[i, "No Finding"]
        
        if p0 < low_threshold:
            # The confidence score is too low
            # Keep, do nothing.
            pass
        
        elif low_threshold <= p0 and p0 < intermediate_threshold:
            # Add, keep "det" preds and add normal pred.
            # It has been shown that this increase the prediction score
            merged_df.loc[i, "PredictionString"] += f" 14 {p0} 0 0 1 1"
            
        elif high_threshold <= p0:
            # When the confidence score is high enough
            # Replace, remove all "det" preds.
            merged_df.loc[i, "PredictionString"] = "14 1 0 0 1 1"
        else:
            pass
    
    # Get the prediction DataFrame after 2-class filter
    ensemble_pred_filter = merged_df[["image_id", "PredictionString"]]
    return ensemble_pred_filter

In [11]:
# Apply the 2-class filter to the ensembled Yolov8 prediction
ensemble_pred_filter = apply_2cls_filter(ensemble_pred_df, pred_2cls)
ensemble_pred_filter.head()

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,3 0.8587 1091 1412 1930 1780 0 0.8381 1422 820...
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,3 0.8483 772 1288 1861 1639 0 0.7149 1121 708 ...


In [12]:
ensemble_pred_filter.to_csv('Submission_ensemble_Yolov8_ResNet101.csv', index=None)